In [1]:
import random
import math

In [2]:
class City:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance_from(self, city):
        # Give difference in x, y
        delta_x_sq = pow(city.x - self.x, 2)
        delta_y_sq = pow(city.y - self.y, 2)
        
        # Calculate shortest path
        distance = math.sqrt(abs(delta_x_sq + delta_y_sq))
        return distance

In [3]:
class Route:
    
    def __init__(self, individual, cities):
        # Get individual's chromosome
        chromosome = individual.chromosome
        # Create route
        self.route = [cities[chromosome[gene_index]] for gene_index in range(len(chromosome))]
        self.distance = 0
    
    # Get route distance
    def get_distance(self):
        if self.distance > 0:
            return self.distance
        
        # Loop over cities in route and calculate route distance
        total_distance = 0
        for city_index in range(len(self.route) - 1):
            total_distance += self.route[city_index].distance_from(self.route[city_index + 1])
        
        total_distance += self.route[len(self.route) - 1].distance_from(self.route[0])
        self.distance = total_distance
        
        return total_distance

In [4]:
class Individual:
    
    def __init__(self, chromosome_length=None, chromosome=None):
        if chromosome_length is not None:
            self.chromosome = self.set_chromosome(chromosome_length)
        elif chromosome is not None:
            self.chromosome = chromosome
        self.fitness = None
    
    # Use every city index
    def set_chromosome(self, chromosome_length):
        chromosome = []
        for gene in range(chromosome_length):
            chromosome.append(gene)
        return chromosome
    
    # Search for a specific integer gene in this individual
    def contains_gene(self, gene):
        for i in range(len(self.chromosome)):
            if self.chromosome[i] == gene:
                return True
        return False

In [5]:
class Population:
    
    def __init__(self, population_size, chromosome_length=None):
        # Set None list for crossover or mutation to set individual
        self.individuals = [None] * population_size
        if chromosome_length is not None:
            self.set_individuals(population_size, chromosome_length)
        self.population_fitness = None
    
    # Initialize individuals in population
    def set_individuals(self, population_size, chromosome_length):
        self.individuals = []
        for _ in xrange(population_size):
            self.individuals.append(Individual(chromosome_length=chromosome_length))
    
    # The offset of the individual you want, sorted by fitness
    # 0 is the strongest, len(population)-1 is the weakest
    def get_fittest(self, offset):
        self.individuals.sort(key=lambda x: x.fitness, reverse=True)
        return self.individuals[offset]
    
    def shuffle(self):
        random.shuffle(self.individuals)

In [6]:
class GeneticAlgorithm:
    
    def __init__(self, population_size, mutation_rate, crossover_rate, elitism_count, tournament_size):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_count = elitism_count
        self.tournament_size = tournament_size
        
    def init_population(self, chromosome_length):
        # Initialize population
        return Population(self.population_size, chromosome_length)
    
    # Calculate fitness for an individual
    def calc_fitness(self, individual, cities):
        # Get fitness
        route = Route(individual, cities)
        fitness = 1 / float(route.get_distance())
        
        # Store fitness
        individual.fitness = fitness
        
        return fitness
    
    # Evaluate the whole population
    def eval_population(self, population, maze):
        population_fitness = 0
        
        # Loop over population evaluating individuals and summing population fitness
        for individual in population.individuals:
            population_fitness += self.calc_fitness(individual, cities)
        
        avg_fitness = float(population_fitness) / len(population.individuals)
        population.population_fitness = avg_fitness
    
    # Check if population has met termination condition
    def termination_condition_met(self, generations_count, max_generations):
        return generations_count > max_generations
    
    # Selects parent for crossover using tournament selection
    def select_parent(self, population):
        # Create tournament
        tournament = Population(self.tournament_size)
        
        # Add random individuals to the tournament
        population.shuffle()
        for i in range(self.tournament_size):
            tournament.individuals[i] = population.individuals[i]
        
        # Return the best
        return tournament.get_fittest(0)
    
    # Ordered crossover mutation
    def crossover_population(self, population):
        # Create new population
        new_population = Population(len(population.individuals))
        
        # Loop over current population by fitness
        for population_index in range(len(population.individuals)):
            # Get parent1
            parent1 = population.get_fittest(population_index)
            
            # Apply crossover to this individual?
            if self.crossover_rate > random.uniform(0, 1) and population_index >= self.elitism_count:
                # Find parent2 with tournament selection
                parent2 = self.select_parent(population)
                
                # Create blank offspring chromosome
                offspring_chromosome = [-1] * len(parent1.chromosome)
                offspring = Individual(chromosome=offspring_chromosome)
                
                # Get subset of parent chromosomes
                substr_pos1 = int(random.uniform(0, 1) * len(parent1.chromosome))
                substr_pos2 = int(random.uniform(0, 1) * len(parent1.chromosome))
                
                # Make the smaller the start and the larger the end
                start_substr = min(substr_pos1, substr_pos2)
                end_substr = max(substr_pos1, substr_pos2)
                
                # Loop and add the sub tour from parent1 to our child
                for i in range(start_substr, end_substr):
                    offspring.chromosome[i] = parent1.chromosome[i]
                
                # Loop through parent2's city tour
                for i in range(len(parent2.chromosome)):
                    parent2_gene = i + end_substr
                    if parent2_gene >= len(parent2.chromosome):
                        parent2_gene -= len(parent2.chromosome)
                    
                    # If offspring doesn't have the city, add it
                    if offspring.contains_gene(parent2.chromosome[parent2_gene]) is False:
                        # Loop to find a spare position in the child's tour
                        for ii in range(len(offspring.chromosome)):
                            # Spare position found, add city
                            if offspring.chromosome[ii] == -1:
                                offspring.chromosome[ii] = parent2.chromosome[parent2_gene]
                                break
                
                # Add child
                new_population.individuals[population_index] = offspring
            else:
                # Add individual to new population without applying crossover
                new_population.individuals[population_index] = parent1
        return new_population
    
    # Apply mutation to population
    def mutate_population(self, population):
        # Initialize new population
        new_population = Population(self.population_size)
        
        # Loop over current population by fitness
        for population_index in range(len(population.individuals)):
            individual = population.get_fittest(population_index)
            
            # Skip mutation if this is an elite individual
            if population_index >= self.elitism_count:
                # print 'Mutating population member ' + str(population_index)
                # Loop over individual's genes
                for gene_index in range(len(individual.chromosome)):
                    # print '\tGene index ' + gene_index
                    # Does this gene need mutation?
                    if self.mutation_rate > random.uniform(0, 1):
                        # Get new gene position
                        new_gene_pos = int(random.uniform(0, 1) * len(individual.chromosome))
                        # Swap genes
                        individual.chromosome[gene_index], individual.chromosome[new_gene_pos] = \
                        individual.chromosome[new_gene_pos], individual.chromosome[gene_index]
            
            # Add individual to population
            new_population.individuals[population_index] = individual
        
        # Return mutated population
        return new_population

In [7]:
max_generations = 3000

# Create cities
num_cities = 100
cities = []

# Loop to create random cities
for city_index in range(num_cities):
    # Generate x, y position
    x_pos = int(100 * random.uniform(0, 1))
    y_pos = int(100 * random.uniform(0, 1))
    
    # Add city
    cities.append(City(x_pos, y_pos))

# initial GA
ga = GeneticAlgorithm(100, 0.001, 0.9, 2, 5)

# Innitialize population
population = ga.init_population(len(cities))

# Evaluate population
ga.eval_population(population, cities)

start_route = Route(population.get_fittest(0), cities)
print "Start Distance: " + str(start_route.get_distance())

# Keep track of current generation
generation = 1
# start evolution loop
while ga.termination_condition_met(generation, max_generations) is False:
    # Print fittest individual from population
    route = Route(population.get_fittest(0), cities)
    print "G" + str(generation) + " Best distance: " + str(route.get_distance())
    
    # Apply crossover
    population = ga.crossover_population(population)
    
    # Apply mutation
    population = ga.mutate_population(population)
    
    # Evaluate population
    ga.eval_population(population, cities)
    
    # Increment the current generation
    generation += 1

print 'Stopped after ' + str(max_generations) + ' generations.'
route = Route(population.get_fittest(0), cities)
print 'Best distance: ' + str(route.get_distance())

Start Distance: 5529.67458597
G1 Best distance: 5529.67458597
G2 Best distance: 5383.75757621
G3 Best distance: 5299.89265189
G4 Best distance: 5281.03373724
G5 Best distance: 5149.28565474
G6 Best distance: 5075.9957257
G7 Best distance: 4996.44486754
G8 Best distance: 4931.32118162
G9 Best distance: 4749.75215089
G10 Best distance: 4682.34152245
G11 Best distance: 4596.51730115
G12 Best distance: 4428.31662821
G13 Best distance: 4292.36004589
G14 Best distance: 4292.36004589
G15 Best distance: 4292.36004589
G16 Best distance: 4056.24713256
G17 Best distance: 4015.43259673
G18 Best distance: 4015.43259673
G19 Best distance: 4005.46239976
G20 Best distance: 3917.12449306
G21 Best distance: 3900.06044219
G22 Best distance: 3896.94407219
G23 Best distance: 3786.44805096
G24 Best distance: 3774.3890235
G25 Best distance: 3723.46252429
G26 Best distance: 3715.36491229
G27 Best distance: 3683.46203449
G28 Best distance: 3613.92653207
G29 Best distance: 3613.92653207
G30 Best distance: 3613.